In [2]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$fix_all) %>%
    select(-Id)

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-SalePrice))), 0)
})

### Passing named list to mutate (and probably other dplyr verbs)
https://community.rstudio.com/t/passing-named-list-to-mutate-and-probably-other-dplyr-verbs/2553/6

In [7]:
fn_df <- tribble(
    ~tran_name, ~tran_defin,
    'log',      function(x) log(x+1),
    'sqrt',     function(x) sqrt(x),
    'invcube',  function(x) x**(1/3)
)

for (row in 1:nrow(fn_df)) {
    fn_df[[row, "tran_defin"]](1) %>% print
}

[1] 0.6931472
[1] 1
[1] 1


In [145]:
combined_dataset %>%
select(-SalePrice) %>%
select_if(is.numeric) %>%
gather(var, x) %>%
filter(!is.na(x)) -> df1

for (row in 1:nrow(fn_df)) {
    df1[fn_df[row, "tran_name"]$tran_name] <- (fn_df[row, "tran_defin"]$tran_defin)[[1]](df1$x)
}

df1 %>% head

var,x,log,sqrt,invcube
LotFrontage,65,4.189655,8.062258,4.020726
LotFrontage,80,4.394449,8.944272,4.308869
LotFrontage,68,4.234107,8.246211,4.081655
LotFrontage,60,4.110874,7.745967,3.914868
LotFrontage,84,4.442651,9.165151,4.379519
LotFrontage,85,4.454347,9.219544,4.396830


In [146]:
df1 %>%
gather(tran, value, -var) %>%
group_by(var, tran) %>%
mutate(
    value_normed = (value - mean(value)) / sd(value)
) %>%
select(-value) %>%
group_by(var, tran, value_normed) %>%
arrange(var, tran, value_normed) %>%
summarise(
    k = n()
) %>%
mutate(
    empirical = cumsum(k) / sum(k),
    theoretical = pnorm(value_normed),
    diff_L2 = k*(empirical - theoretical)**2
) %>%
group_by(var, tran) %>%
summarise(
    L2_distance = sum(diff_L2)
) -> df2

df2 %>% head

var,tran,L2_distance
BedroomAbvGr,invcube,106.21667
BedroomAbvGr,log,104.29266
BedroomAbvGr,sqrt,111.96468
BedroomAbvGr,x,134.35462
BsmtFinSF1,invcube,66.20057
BsmtFinSF1,log,95.80909


In [153]:
df2 %>%
group_by(var) %>%
nest %>%
mutate(
    best_tran = map(data, function(df) {
        best <- df %>% arrange(L2_distance) %>% head(1)
        vanilla <- df %>% filter(tran == 'x')
        progress_score <- 100 * (vanilla$L2_distance - best$L2_distance) / vanilla$L2_distance
        data_frame(tran = best$tran, progress_score = progress_score)
    })
) %>%
select(var, best_tran) %>%
unnest(best_tran) %>%
filter(tran != 'x') %>%
arrange(desc(progress_score)) %>%
inner_join(fn_df, by=c("tran" = "tran_name")) -> transformation_config

transformation_config

var,tran,progress_score,tran_defin
GrLivArea,log,91.15794157,"function (x) , log(x + 1)"
X1stFlrSF,log,90.77165686,"function (x) , log(x + 1)"
BsmtUnfSF,sqrt,79.97603070,"function (x) , sqrt(x)"
LotArea,invcube,78.43689082,"function (x) , x^(1/3)"
TotRmsAbvGrd,log,30.68501506,"function (x) , log(x + 1)"
OverallQual,log,29.94435652,"function (x) , log(x + 1)"
GarageCars,log,24.95676474,"function (x) , log(x + 1)"
BedroomAbvGr,log,22.37508601,"function (x) , log(x + 1)"
OverallCond,log,16.24723935,"function (x) , log(x + 1)"
FullBath,invcube,8.79733251,"function (x) , x^(1/3)"


In [167]:
combined_dataset %>%
select(one_of(transformation_config$var)) %>%
gather(var, value) %>%
filter(!is.na(value)) %>%
inner_join(transformation_config %>% select(var, tran, tran_defin), by='var') %>%
head(2) %>%
mutate(
    value_transformed = map2_dbl(value, tran_defin, function(val, fn) fn(val))
) %>%
select(var, value, value_transformed) %>%
gather(tran, value, -var) %>%
mutate(
    tran = ifelse(tran == 'value', 'original', 'transformed')
)

var,tran,value
GrLivArea,original,1710.000000
GrLivArea,original,1262.000000
GrLivArea,transformed,7.444833
GrLivArea,transformed,7.141245
